In [2]:
import torch
import cv2
import numpy as np
import datetime
from shapely.geometry import Polygon

In [3]:
# MPII에서 각 파트 번호, 선으로 연결될 POSE_PAIRS
BODY_PARTS = { "Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
                "Background": 15 }

POSE_PAIRS = [ ["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder", "RElbow"],
                ["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder", "LElbow"],
                ["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"], ["RHip", "RKnee"],
                ["RKnee", "RAnkle"], ["Chest", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"] ]

In [4]:
# 각 파일 path
protoFile = "./openpose-master/models/pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "./openpose-master/models/pose/mpi/pose_iter_160000.caffemodel"

In [5]:
# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

In [6]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_path = "./sample_video/sample2.mp4"
cap = cv2.VideoCapture(video_path)
video_w, video_h, video_fps = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), float(cap.get(cv2.CAP_PROP_FPS))
video = cv2.VideoWriter("./result/" + "openpose_result1.avi", fourcc, video_fps, (video_w, video_h))


In [7]:
inputWidth=320;
inputHeight=240;
inputScale=1.0/255;

In [8]:
def mosaic(img, points, partA, partB, ratio=0.1):
    x1, y1 = points[partA]
    x2, y2 = points[partB]

    # 선 그어진 부분의 중심 좌표
    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

    # 선 길이 계산
    length = int(np.sqrt((x2 - x1)**2 + (y2 - y1)**2))

    # 모자이크 처리를 위한 사각형의 시작, 끝 좌표 계산
    x_start, y_start = cx - length // 2, cy - length // 2
    x_end, y_end = cx + length // 2, cy + length // 2

    # 모자이크 처리
    roi = img[y_start:y_end, x_start:x_end]
    roi = cv2.resize(roi, (0, 0), fx=ratio, fy=ratio)
    roi = cv2.resize(roi, (length, length), interpolation=cv2.INTER_AREA)
    img[y_start:y_end, x_start:x_end] = roi

In [ ]:
#반복문을 통해 카메라에서 프레임을 지속적으로 받아옴
while True:  #아무 키나 누르면 끝난다.
   
    hasFrame, frame = cap.read()  
    
    #영상이 커서 느리면 사이즈를 줄이자
    #frame=cv2.resize(frame,dsize=(320,240),interpolation=cv2.INTER_AREA)
    
    # 
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    inpBlob = cv2.dnn.blobFromImage(frame, inputScale, (inputWidth, inputHeight), (0, 0, 0), swapRB=False, crop=False)
    
    imgb=cv2.dnn.imagesFromBlob(inpBlob)
    #cv2.imshow("motion",(imgb[0]*255.0).astype(np.uint8))
    
    # network에 넣어주기
    net.setInput(inpBlob)

    # 결과 받아오기
    output = net.forward()


    # 키포인트 검출시 이미지에 그려줌
    points = []
    for i in range(0,15):
        # 해당 신체부위 신뢰도 얻음.
        probMap = output[0, i, :, :]
    
        # global 최대값 찾기
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # 원래 이미지에 맞게 점 위치 변경
        x = (frameWidth * point[0]) / output.shape[3]
        y = (frameHeight * point[1]) / output.shape[2]

        # 키포인트 검출한 결과가 0.1보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가, 검출했는데 부위가 없으면 None으로    
        if prob > 0.1 :    
            cv2.circle(frame, (int(x), int(y)), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED) # circle(그릴곳, 원의 중심, 반지름, 색)
            cv2.putText(frame, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, lineType=cv2.LINE_AA)
            points.append((int(x), int(y)))
        else :
            points.append(None)
    
    

    # 각 POSE_PAIRS별로 선 그어줌 (머리 - 목, 목 - 왼쪽어깨, ...)
    for pair in POSE_PAIRS:
        partA = pair[0]             # Head
        partA = BODY_PARTS[partA]   # 0
        partB = pair[1]             # Neck
        partB = BODY_PARTS[partB]   # 1
        
        #partA와 partB 사이에 선을 그어줌 (cv2.line)
        if points[partA] and points[partB]:
            cv2.line(frame, points[partA], points[partB], (0, 255, 0), 2)
            mosaic(frame, points, partA, partB)
        
            
    cv2.imshow("Output-Keypoints",frame)



cap.release()  #카메라 장치에서 받아온 메모리 해제
cv2.destroyAllWindows() #모든 윈도우 창 닫음

: 